In [1]:
import os
import sys
import glob
import pathlib
import yaml
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import argparse
from sklearn.model_selection import KFold, train_test_split
from torchvision import transforms
from torchvision.models.resnet import resnet18, resnet50, ResNet50_Weights
from dataloader import MedicalData
from trainer import Trainer
from datetime import datetime
from utils import *
import albumentations as A

In [2]:
cfg_path = './config/config.yml'
cfg = load_config(cfg_path)

In [3]:
df = load_data(cfg["datasets"]["root"])
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [4]:
cfg['trainer']['epochs'] = 30

In [5]:
device = check_gpu(cfg['use_gpu'])

In [6]:
model_dir, visual_dir = create_folder()

In [7]:
def setup_dataflow(train_idx, val_idx):
    root = cfg['datasets']['root']
    width = cfg['datasets']['image_width']
    height = cfg['datasets']['image_height']
    batch_size = cfg['datasets']['batch_size']
    classes = pd.read_csv('./class.csv')
    
    

    transform_train = A.Compose([
        A.Resize(224, 224),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.ShiftScaleRotate(p=0.5),
        A.RandomCrop(p=0.2, height=224, width=224),
        A.Normalize(),
    ])
    
    transform_valid = A.Compose([
        A.Resize(224, 224),
        A.Normalize(),
    ])
    
    train_sampler = MedicalData(root, train_idx, classes, device, transform_train)
    val_sampler = MedicalData(root, val_idx, classes, device, transform_valid)

    train_loader = torch.utils.data.DataLoader(train_sampler, batch_size=batch_size)
    val_loader = torch.utils.data.DataLoader(val_sampler, batch_size=batch_size)

    return train_loader, val_loader

In [8]:
n_splits = 5
kfold = KFold(n_splits, shuffle=True, random_state=42)

In [9]:
def load_model():
    model = resnet50(ResNet50_Weights)
    model.fc = nn.Linear(2048, cfg['model']['num_classes'])
    for idx, child in enumerate(model.children()):
        if idx < 6:
            for param in child.parameters():
                param.requires_grad = False
    return model

In [10]:
torch.cuda.empty_cache()

In [11]:
for i, (train_idx, valid_idx) in enumerate(kfold.split(df)):
    print('===' * 10 + "{}".format(i+1) + "===" * 10)
    df_train = df.iloc[train_idx]
    df_valid = df.iloc[valid_idx]
    train_loader, valid_loader = setup_dataflow(df_train, df_valid)
    test_loader = None
    model = load_model()
    if i >= 4:
        trainer = Trainer(model=model, 
                            config=cfg,
                            train_loader=train_loader,
                            valid_loader=valid_loader,
                            test_loader=test_loader,
                            device=device)

        trainer.train(kfold=i+1, save_dir=model_dir)
        trainer.visualize(visual_dir, i+1)

==============================1==============================


/home/hovduc/miniconda3/envs/medical_env/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/hovduc/miniconda3/envs/medical_env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


==============================2==============================


/home/hovduc/miniconda3/envs/medical_env/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/hovduc/miniconda3/envs/medical_env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


==============================3==============================


...

Saved!
EPOCH: 15/30


  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/118 [00:00<?, ?it/s]

              precision    recall  f1-score   support

      ASC_US       0.51      0.59      0.55       682
        LSIL       0.66      0.49      0.56       835
         SCC       0.73      0.83      0.78       373
        HSIL       0.60      0.42      0.49       972
       ASC_H       0.44      0.68      0.53       760
         AIS       0.76      0.45      0.57       128

    accuracy                           0.56      3750
   macro avg       0.62      0.58      0.58      3750
weighted avg       0.58      0.56      0.56      3750

metrics: {'loss': 0.48384745762711884, 'accuracy': 0.562101694915254, 'f1_scores': 0.562101694915254, 'precision': 0.5460084745762708, 'mAP': 0.5207372881355932, 'recall': 0.5463813559322036}
Confusion Matrix:
 tensor([[403., 173.,   9.,  23.,  74.,   0.],
        [271., 410.,  13.,  40.,  97.,   4.],
        [  0.,   1., 310.,  40.,  22.,   0.],
        [ 40.,  21.,  52., 407., 442.,  10.],
        [ 78.,  19.,  26., 120., 513.,   4.],
        [  0.,  

  0%|          | 0/469 [00:00<?, ?it/s]

In [ ]:
cfg_copy = cfg
cfg_copy['trainer']['epochs'] = 1
cfg_copy['trainer']['print_every'] = 1
cfg_copy['trainer']['lr'] = 1e-4

In [ ]:
model.load_state_dict(torch.load('./save/save_2023-06-14 10:56:24.952644/model_fold5_epoch4.pt'))
df_train = df.iloc[train_idx]
df_valid = df.iloc[valid_idx]
train_loader, valid_loader = setup_dataflow(df_train, df_valid)
test_loader = None
trainer = Trainer(model=model, 
                    config=cfg_copy,
                    train_loader=train_loader,
                    valid_loader=valid_loader,
                    test_loader=test_loader,
                    device=device)

trainer.train(kfold=i+1, save_dir=model_dir)

RuntimeError: Error(s) in loading state_dict for ResNet:
	Missing key(s) in state_dict: "layer1.0.conv3.weight", "layer1.0.bn3.weight", "layer1.0.bn3.bias", "layer1.0.bn3.running_mean", "layer1.0.bn3.running_var", "layer1.0.downsample.0.weight", "layer1.0.downsample.1.weight", "layer1.0.downsample.1.bias", "layer1.0.downsample.1.running_mean", "layer1.0.downsample.1.running_var", "layer1.1.conv3.weight", "layer1.1.bn3.weight", "layer1.1.bn3.bias", "layer1.1.bn3.running_mean", "layer1.1.bn3.running_var", "layer1.2.conv1.weight", "layer1.2.bn1.weight", "layer1.2.bn1.bias", "layer1.2.bn1.running_mean", "layer1.2.bn1.running_var", "layer1.2.conv2.weight", "layer1.2.bn2.weight", "layer1.2.bn2.bias", "layer1.2.bn2.running_mean", "layer1.2.bn2.running_var", "layer1.2.conv3.weight", "layer1.2.bn3.weight", "layer1.2.bn3.bias", "layer1.2.bn3.running_mean", "layer1.2.bn3.running_var", "layer2.0.conv3.weight", "layer2.0.bn3.weight", "layer2.0.bn3.bias", "layer2.0.bn3.running_mean", "layer2.0.bn3.r

In [ ]:
df_train.to_csv('./data/train.csv')
df_valid.to_csv('./data/valid.csv')